In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import grab_weather_stats
import os
import requests
import json

pd.set_option('display.max_columns',None)
scaler = StandardScaler()

In [7]:
flight_df = pd.read_feather('data/v2_clean_flight')
flight_test_df = pd.read_feather('data/v1_clean_flight_test')
fuel_consumption_df = pd.read_feather('data/v1_clean_fuel_consumption')
passenger_df = pd.read_feather('data/v1_clean_passenger')

In [8]:
def duration_to_rank(x):
    """ 
    Transforms the rank of a flight distance

    Parameters:
        x(str): short,medium or long flight distance

    Returns:
        An integer of 0, 1 or 2 depending on their flight distance
    """
    if x == 'short':
        return 0
    elif x =='medium':
        return 1
    else:
        return 2

flight_df['flight_duration'] = flight_df['flight_duration'].apply(lambda x: duration_to_rank(x))

In [11]:
num_cols = ['crs_elapsed_time','distance']
rank_cols = ['month','flight_duration','hour']
dum_cols = ['origin','dest']
y_target = flight_df['arr_delay'].fillna(0)

In [ ]:
flight_model = flight_df[['crs_elapsed_time','distance','month','flight_duration','hour','origin','dest']]
flight_model = pd.get_dummies(flight_model, columns=dum_cols)
flight_model

In [13]:
x_train,x_test,y_train,y_test = train_test_split(flight_model,y_target,test_size=0.3)

In [14]:
x_train_scaled = scaler.fit_transform(x_train[num_cols])
x_test_scaled = scaler.fit_transform(x_test[num_cols])
x_train[num_cols] = x_train_scaled
x_test[num_cols] = x_test_scaled

In [16]:
## Took 67minutes to run this fit. best params was squared error, max_depth 2, n_estimators = 50

# param_grid = {
#     'max_depth': [2,3,4,5,6],
#     'criterion': ['absoulate_error','squared_error'],
#     'n_estimators': [10,30,50,70,100]
# }
# rforest = RandomForestRegressor()
# regCV = GridSearchCV(estimator=rforest,cv=5,param_grid=param_grid,return_train_score=True)
# regCV.fit(x_train,y_train)
# regCV.best_params_ (best params depth 2, n_est 50)

In [ ]:
regressor = RandomForestRegressor(random_state=0,max_depth=2,n_estimators=50)
regressor.fit(x_train,y_train)
y_pred = regressor.predict(x_test)
r2_score(y_test,y_pred)

# 500 Entries of weather stats for model

Only used 500 entries of weather data, did not have any good value adding it into the model

In [ ]:
# weather_flight_model = flight_df[:500].copy()
# weather_flight_model

In [ ]:
# lat_list,lon_list = grab_weather_stats.lat_grabber(weather_flight_model)

In [ ]:
# weather_flight_model['lat'] = lat_list
# weather_flight_model['lon'] = lon_list

In [ ]:
# cloud,rain,snow,sunny = grab_weather_stats.weather_grabber(weather_flight_model)

In [67]:
# small_weather_model = weather_flight_model[:500].copy()
# small_weather_model['cloud'] = cloud
# small_weather_model['sunny'] = sunny
# small_weather_model['snow'] = snow
# small_weather_model['rain'] = rain

In [69]:
#small_weather_model.to_feather('data/flight_weather_model_500')

In [ ]:
# num_cols = ['crs_elapsed_time','distance','cloud','sunny','rain','snow']
# rank_cols = ['month','flight_duration','hour']
# dum_cols = ['origin','dest']
# y_target_weather = flight_df['arr_delay'].fillna(0)[:500]
# small_weather_model = small_weather_model[['crs_elapsed_time','distance','cloud','sunny','rain','snow','month','flight_duration','hour','origin','dest']]
# small_weather_model

In [ ]:
# small_weather_model = pd.get_dummies(small_weather_model,columns=dum_cols)
# small_weather_model

In [61]:
# x_train,x_test,y_train,y_test = train_test_split(small_weather_model,y_target_weather,test_size=0.3)
# x_train_scaled = scaler.fit_transform(x_train[num_cols])
# x_test_scaled = scaler.fit_transform(x_test[num_cols])

In [62]:
# x_train[num_cols] = x_train_scaled
# x_test[num_cols] = x_test_scaled
# regressor = RandomForestRegressor(n_estimators=100,random_state=0)
# regressor.fit(x_train,y_train)
# y_pred = regressor.predict(x_test)

In [63]:
# r2_score(y_test,y_pred)

-0.2661480866278345